In [42]:
import pandas as pd 
import gspread
import os
from oauth2client.service_account import ServiceAccountCredentials
import mysql.connector
from datetime import datetime
import pendulum
import pandas as pd
import numpy as np 
from dotenv import load_dotenv
from forex_python.converter import CurrencyRates
import requests



In [43]:

load_dotenv()

password = os.getenv("MYSQL_PASSWORD")
conn = mysql.connector.connect(
    host="localhost",
    user ="root",
    password=password,
    database= "job_project"
)

students_df = pd.read_sql("SELECT * FROM students", conn)
lessons_df = pd.read_sql("SELECT * FROM lesson", conn)
conn.close()


/var/folders/1k/fr48krgd145fj258_ll4wx1m0000gn/T/ipykernel_10659/3376892106.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  students_df = pd.read_sql("SELECT * FROM students", conn)
/var/folders/1k/fr48krgd145fj258_ll4wx1m0000gn/T/ipykernel_10659/3376892106.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  lessons_df = pd.read_sql("SELECT * FROM lesson", conn)


In [44]:
print (students_df)

    student_id student_name  time       fee currency
0            1         크세니야  1.00   1400.00      RUB
1            2        아나스타샤  0.75    900.00      RUB
2            3           사라  0.75     14.45      USD
3            4           싸샤  1.00     15.30      USD
4            5         세르게이  1.00  22000.00      KRW
5            6     나탈리아 (상)  1.00     15.30      USD
6            7           엠마  0.50      8.50      USD
7            8           막심  1.00     15.30      USD
8            9         발레리아  1.00   1300.00      RUB
9           10          아르쬼  0.75   1300.00      RUB
10          11           샐리  1.00     12.75      USD
11          12           헬가  1.00     15.30      USD
12          13          디아나  1.00   1500.00      RUB
13          14          엘레나  1.00  19000.00      KRW
14          15          비탈리  0.75      9.35      USD
15          16          이리나  1.00     15.30      USD
16          17         베로니카  1.00     15.30      USD
17          18     나탈리아 (한)  1.00  20000.00   

In [45]:
print(lessons_df)

    student_id lesson_date   lesson_type student_name
0            1  2024-10-13       grammar         크세니야
1            1  2024-10-20       grammar         크세니야
2            2  2024-10-06       reading        아나스타샤
3            2  2024-10-13       reading        아나스타샤
4            3  2024-10-02         topik           사라
..         ...         ...           ...          ...
77          39  2024-10-16       grammar     크세니야 (브)
78          40  2024-10-06       grammar           노라
79          41  2024-10-08  conversation          앤드류
80          42  2024-10-21       grammar          새학생
81          42  2024-10-22       grammar          새학생

[82 rows x 4 columns]


In [46]:
merged_df = pd.merge(lessons_df, students_df, on='student_id')
print(merged_df.head(5))

   student_id lesson_date lesson_type student_name_x student_name_y  time  \
0           1  2024-10-13     grammar           크세니야           크세니야  1.00   
1           1  2024-10-20     grammar           크세니야           크세니야  1.00   
2           2  2024-10-06     reading          아나스타샤          아나스타샤  0.75   
3           2  2024-10-13     reading          아나스타샤          아나스타샤  0.75   
4           3  2024-10-02       topik             사라             사라  0.75   

       fee currency  
0  1400.00      RUB  
1  1400.00      RUB  
2   900.00      RUB  
3   900.00      RUB  
4    14.45      USD  


In [47]:
merged_df = merged_df.drop('student_name_y', axis=1)
print(merged_df)

    student_id lesson_date   lesson_type student_name_x  time      fee  \
0            1  2024-10-13       grammar           크세니야  1.00  1400.00   
1            1  2024-10-20       grammar           크세니야  1.00  1400.00   
2            2  2024-10-06       reading          아나스타샤  0.75   900.00   
3            2  2024-10-13       reading          아나스타샤  0.75   900.00   
4            3  2024-10-02         topik             사라  0.75    14.45   
..         ...         ...           ...            ...   ...      ...   
77          39  2024-10-16       grammar       크세니야 (브)  0.75    13.60   
78          40  2024-10-06       grammar             노라  0.75    13.60   
79          41  2024-10-08  conversation            앤드류  1.00    12.75   
80          42  2024-10-21       grammar            새학생  0.50     8.00   
81          42  2024-10-22       grammar            새학생  0.50     8.00   

   currency  
0       RUB  
1       RUB  
2       RUB  
3       RUB  
4       USD  
..      ...  
77      USD  

In [48]:
oct_2_data = merged_df.loc[merged_df['lesson_date'] == '2024-10-02']
print(oct_2_data)

    student_id lesson_date   lesson_type student_name_x  time    fee currency
4            3  2024-10-02         topik             사라  0.75  14.45      USD
22          11  2024-10-02  conversation             샐리  1.00  12.75      USD
46          21  2024-10-02       grammar             안나  1.00  15.30      USD
61          29  2024-10-02         topik             올랴  1.00  17.00      USD
63          30  2024-10-02       grammar            폴리나  0.75  13.60      USD
66          32  2024-10-02       grammar            레기나  0.75  13.60      USD
73          37  2024-10-02  conversation             상호  1.00  12.75      USD


In [49]:

merged_df['lesson_date'] = pd.to_datetime(merged_df['lesson_date'], errors='coerce')
print(merged_df.dtypes)

student_id                 int64
lesson_date       datetime64[ns]
lesson_type               object
student_name_x            object
time                     float64
fee                      float64
currency                  object
dtype: object


In [50]:
api_key = os.getenv("api_key")
url = f"http://data.fixer.io/api/latest?access_key={api_key}"
response = requests.get(url)
data = response.json()


In [51]:
eur_to_usd = data['rates']['USD']  # 1 EUR -> USD 환율
eur_to_rub = data['rates']['RUB']  # 1 EUR -> RUB
eur_to_krw = data['rates']['KRW'] # 1EUR -> KRW
EUR = data['rates']['EUR']

usd_to_eur = 1 / eur_to_usd
rub_to_eur = 1 / eur_to_rub
usd_to_krw = usd_to_eur * eur_to_krw
rub_to_krw = rub_to_eur * eur_to_krw

print(usd_to_eur)
print(rub_to_eur)
print(usd_to_krw)
print(rub_to_krw)



0.9258307710966743
0.00966888805824546
1380.6503109402645
14.41878334660501


## Daily revenue


In [52]:
daily_revenue = merged_df.groupby(['lesson_date','currency']).apply(lambda x: (x['fee']).sum()).unstack()
print(daily_revenue)

currency         KRW     RUB    USD
lesson_date                        
2024-10-02       NaN     NaN  99.45
2024-10-03   19000.0  1300.0  33.15
2024-10-04       NaN  3000.0  28.90
2024-10-06   22000.0   900.0  39.95
2024-10-07   19000.0  4100.0  41.65
2024-10-08       NaN     NaN  25.50
2024-10-09       NaN  1500.0  81.60
2024-10-10   19000.0  4300.0  15.30
2024-10-11       NaN     NaN  15.30
2024-10-13   22000.0  2300.0  63.75
2024-10-14   19000.0  2800.0  28.05
2024-10-16   20000.0  1500.0  71.40
2024-10-17   19000.0     NaN  32.30
2024-10-18       NaN  3000.0  26.35
2024-10-20   22000.0  1400.0  11.05
2024-10-21   19000.0  2800.0  28.40
2024-10-22       NaN     NaN  34.35


/var/folders/1k/fr48krgd145fj258_ll4wx1m0000gn/T/ipykernel_10659/1584301517.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  daily_revenue = merged_df.groupby(['lesson_date','currency']).apply(lambda x: (x['fee']).sum()).unstack()


In [53]:
daily_revenue = merged_df.groupby(['lesson_date','currency']).apply(lambda x: (x['fee']).sum()).unstack()
daily_revenue['total_eur'] = (daily_revenue['USD'].fillna(0) * usd_to_eur + daily_revenue['RUB'].fillna(0) * rub_to_eur).round(2)
daily_revenue['total_krw'] = (daily_revenue['KRW'].fillna(0) + daily_revenue['USD'].fillna(0) * usd_to_krw + daily_revenue['RUB'].fillna(0) * rub_to_krw).round(2)


print(daily_revenue)

currency         KRW     RUB    USD  total_eur  total_krw
lesson_date                                              
2024-10-02       NaN     NaN  99.45      92.07  137305.67
2024-10-03   19000.0  1300.0  33.15      43.26   83512.98
2024-10-04       NaN  3000.0  28.90      55.76   83157.14
2024-10-06   22000.0   900.0  39.95      45.69   90133.88
2024-10-07   19000.0  4100.0  41.65      78.20  135621.10
2024-10-08       NaN     NaN  25.50      23.61   35206.58
2024-10-09       NaN  1500.0  81.60      90.05  134289.24
2024-10-10   19000.0  4300.0  15.30      55.74  102124.72
2024-10-11       NaN     NaN  15.30      14.17   21123.95
2024-10-13   22000.0  2300.0  63.75      81.26  143179.66
2024-10-14   19000.0  2800.0  28.05      53.04   98099.83
2024-10-16   20000.0  1500.0  71.40      80.61  140206.61
2024-10-17   19000.0     NaN  32.30      29.90   63595.01
2024-10-18       NaN  3000.0  26.35      53.40   79636.49
2024-10-20   22000.0  1400.0  11.05      23.77   57442.48
2024-10-21   1

/var/folders/1k/fr48krgd145fj258_ll4wx1m0000gn/T/ipykernel_10659/3855049484.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  daily_revenue = merged_df.groupby(['lesson_date','currency']).apply(lambda x: (x['fee']).sum()).unstack()


## Weekly Revenue

In [54]:
daily_revenue = daily_revenue.reset_index()
daily_revenue['lesson_date'] = pd.to_datetime(daily_revenue['lesson_date'])

weekly_revenue = daily_revenue.groupby(pd.Grouper(key='lesson_date', freq='W-SUN')).sum()

print(weekly_revenue)

currency         KRW      RUB     USD  total_eur  total_krw
lesson_date                                                
2024-10-06   41000.0   5200.0  201.45     236.78  394109.67
2024-10-13   60000.0  12200.0  243.10     343.03  571545.25
2024-10-20   80000.0   8700.0  169.15     240.72  438980.42
2024-10-27   19000.0   2800.0   62.75      85.17  146008.40


## Monthly Revenue

In [56]:

daily_revenue = daily_revenue.reset_index()
daily_revenue['lesson_date'] = pd.to_datetime(daily_revenue['lesson_date'])

# 월별 수입 계산 (이번 달도 포함)
current_date = pd.to_datetime('today')

monthly_revenue = daily_revenue[daily_revenue['lesson_date'] <= current_date].groupby(
pd.Grouper(key='lesson_date', freq='M')
).sum()

print(monthly_revenue)


currency     level_0  index       KRW      RUB     USD  total_eur   total_krw
lesson_date                                                                  
2024-10-31       136    136  200000.0  28900.0  676.45      905.7  1550643.74


/var/folders/1k/fr48krgd145fj258_ll4wx1m0000gn/T/ipykernel_10659/3841289007.py:8: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  pd.Grouper(key='lesson_date', freq='M')
